In [15]:
import requests
import os
import pandas as pd
import json
from time import sleep
from dotenv import load_dotenv

In [16]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [55]:
LISTE_RESEAUX_ETUDIES = ["DB SNCF",
                         "Eurostar",
                         "Intercit\u00e9s",
                         "Intercit\u00e9s de nuit",
                         "LEX",
                         "OUIGO","OUIGO Train Classique",
                         "SNCF",
                         "TER HDF",
                         "TER NA",
                         "TGV INOUI",
                         "TGV Lyria",
                         "TRANSILIEN",
                         "additional service",
                         "TER"]

In [10]:
def requete_reseau(code,api_key):
    '''Requete une liste de gare à une page spécifique'''
    reponse = requests.get(f"https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:SNCF:{code}/networks", auth=(api_key,""))
    sleep(3)
    return reponse.json()

In [2]:
def enregistrer_json(data,nom_fichier):
    '''Cette fonction enregistre un fichier json'''
    with open(nom_fichier,"w") as fc:
        json.dump(data,fc)

In [3]:
def dataframe_to_csv(dataframe,nomfichier):
    '''Cette fonction enregistre un dataframe en csv'''
    dataframe.to_csv(nomfichier)

In [6]:
def joindre_dataframe(df1,df2):
    '''Cette fonction permet de joindre deux dataframe'''
    df3 = pd.merge(df1,df2, how='inner', on="id")
    return df3

In [7]:
def top100(df):
    '''Cette fonction classe un dataframe selon la frequentation des gare renvoie les 100 premières valeurs'''

    #Range les enregistrements en fonction du nombre de voyageur
    df_ranked = df.sort_values("total_voyageurs_2022", ascending=False)
    #Les 100 premières valeur
    df_100 = df_ranked.iloc[:100]

    return df_100   

In [45]:
def ajout_reseau(data,liste_gares,cle_api):
    '''Creer une liste de gare en ajoutant un dictionnaire de réseau à chaque gare'''
    for item in data:
        # Extraction du code uic
        code_gare = item["code_uic"]
        # Requete pour récupérer les infos réseau
        resultat = requete_reseau(code_gare,cle_api)
        # Extraction de tous les reseau d'une gare
        tous_reseaux = resultat["networks"]
        # Initialisation d'une liste de réseau
        liste_reseaux = []
        # Ajout du nom de chaque réseau à la liste des réseaux
        for reseau in tous_reseaux:
            liste_reseaux.append(reseau["name"])
        # Ajout d'un ensemble clé valeur avec la liste des réseaux
        item["network"]=liste_reseaux
        # Ecriture de toutes les informations de la gare dans une liste
        liste_gares.append(item)
    
    return liste_gares


In [53]:
def trie_gare(data,LISTE_RESEAUX_ETUDIES):

    liste_inclue = []
    liste_exclue = []

    for gare in data:
        if any(reseau in gare["network"] for reseau in LISTE_RESEAUX_ETUDIES):
            liste_inclue.append(gare)
        else:
            liste_exclue.append(gare)
    
    enregistrer_json(liste_inclue,"data/gare_frequentation_inclues.json")
    enregistrer_json(liste_exclue,"data/gare_frequentation_exclues.json")

In [54]:
with open("data/donnees_brutes_frequentation.json", "r") as f:
    data = json.load(f)


data_test = data[:20]
liste_gares = []
gares_reseaux = ajout_reseau(data_test,liste_gares,api_key)

enregistrer_json(gares_reseaux,"data/test.json")
trie_gare(gares_reseaux,LISTE_RESEAUX_ETUDIES)